In [1]:
# Define environment variables 
%env DEEP_ROOT=/content/drive/My Drive/Colab/Deep/brAIn-Power
%env DEEP_DATA=/content/drive/My Drive/Colab/Deep/DeepLearning/correct_data

# Connect to drive
from google.colab import drive
drive.mount('/content/drive')

# Change directory before importing files
import os
deep_root = os.getenv("DEEP_ROOT")
%cd $deep_root

!pip install import-ipynb
import import_ipynb

env: DEEP_ROOT=/content/drive/My Drive/Colab/Deep/brAIn-Power
env: DEEP_DATA=/content/drive/My Drive/Colab/Deep/DeepLearning/correct_data
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab/Deep/brAIn-Power


In [2]:
from LoadData import CustomDataset, load_data_2np
import UNet
import torch
import time
import numpy as np
from torch.utils.data import DataLoader,Dataset

importing Jupyter notebook from LoadData.ipynb
importing Jupyter notebook from UNet.ipynb


In [3]:
# Define some helper functions for later:
def IoU_score(target, prediction):
    intersection = np.logical_and(target, prediction)
    union = np.logical_or(target, prediction)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

Load in data

In [4]:
X_train, Y_train, X_valid, Y_valid, X_trans, Y_trans, X_test, Y_test = load_data_2np(hot_encoding=True, valid_perc=0.1, train_perc=0.1,show=0,CAD_perc=0,background=True)
DS_train = CustomDataset(X_train, Y_train,one_hot=True)
DS_valid = CustomDataset(X_valid, Y_valid,one_hot=True)
DS_test = CustomDataset(X_test,Y_test,one_hot=True)

print(X_train.shape,Y_train.shape)

(12, 3, 256, 256) (12, 9, 256, 256)


Perform preprocessing:

In [5]:
#DS_train.gray_gamma_enhanced()
#DS_valid.gray_gamma_enhanced()
#DS_test.gray_gamma_enhanced()

Check if GPU is available and functions to convert Tensors to Cuda

In [6]:
use_cuda = torch.cuda.is_available()
print("Running GPU.") if use_cuda else print("No GPU available.")

Running GPU.


In [7]:
#load in model
%%capture
net = UNet.UNet18(n_classes=9, rgb=True)
net.double()
if use_cuda:
    print('converting network to cuda-enabled')
    net.cuda()


In [8]:
def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x

In [9]:
def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

Create lossfunction and optimizer

In [10]:
criterion = UNet.DiceLoss()
optimizer = torch.optim.SGD(net.parameters(),lr=1e-3)

Define hyper parameters

In [11]:
NUM_EPOCHS = 10
check_at = 1

Define list to stor intermediat results

In [15]:
valid_iter = []
valid_loss = []
valid_iou = []

train_iter = []
train_loss = []
train_iou = []

# Define data loaders to serve data
DL_train = DataLoader(DS_train, batch_size=4, shuffle=True)
DL_valid = DataLoader(DS_valid)

#training loop
for epoch in range(NUM_EPOCHS):
    net.train()
    for i,data in enumerate(DL_train):
        input = data[0].type(torch.DoubleTensor)
        target = data[1].type(torch.DoubleTensor)

        # Tranfer to GPU if possible
        input = get_variable(input)
        target = get_variable(target)

        #train the network
        optimizer.zero_grad()
        output = net(input)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()

        #store the results
        train_iter.append(epoch*len(DL_train)+i)
        train_loss.append(loss)

    #validate
    if epoch%check_at==0:
        net.eval()
        L = 0
        for j,valid_data in enumerate(DL_valid):
            input = valid_data[0].type(torch.DoubleTensor)
            target = valid_data[1].type(torch.DoubleTensor)
            
            # Tranfer to GPU if possible
            input = get_variable(input)
            target = get_variable(target)
            
            output = net(input)
            loss = criterion(output, target)
            L += loss
            valid_iou.append(IoU_score(get_numpy(target),get_numpy(output)))
        valid_iter.append(epoch)
        valid_loss.append(L/j)
        print(f'At epoch {epoch} Training loss is at {train_loss[-1]}')
        print(f'At epoch {epoch} Validation loss is at {valid_loss[-1]} and the IoU is at {valid_iou[-1]}%')

At epoch 0 Training loss is at 0.8163508553264147
At epoch 0 Validation loss is at inf and the IoU is at 0.1111111111111111%
At epoch 1 Training loss is at 0.8161520695424187
At epoch 1 Validation loss is at inf and the IoU is at 0.1111111111111111%
At epoch 2 Training loss is at 0.8161250391220709
At epoch 2 Validation loss is at inf and the IoU is at 0.1111111111111111%
At epoch 3 Training loss is at 0.8163013801587559
At epoch 3 Validation loss is at inf and the IoU is at 0.1111111111111111%
At epoch 4 Training loss is at 0.8161490836401866
At epoch 4 Validation loss is at inf and the IoU is at 0.1111111111111111%
At epoch 5 Training loss is at 0.8161238298557973
At epoch 5 Validation loss is at inf and the IoU is at 0.1111111111111111%
At epoch 6 Training loss is at 0.8161576750871544
At epoch 6 Validation loss is at inf and the IoU is at 0.1111111111111111%
At epoch 7 Training loss is at 0.8163192021590708
At epoch 7 Validation loss is at inf and the IoU is at 0.1111111111111111%
